In [3]:
import cv2 as cv
import numpy as np

inputMode = True
roiPts = [] 
frame = None
roiBox = None

# callback del mouse
def selectROI(event, x, y, flags, param):
    global frame, roiPts, inputMode
    if inputMode and event==cv.EVENT_LBUTTONDOWN and len(roiPts) < 4:
        roiPts.append((x, y))
        cv.circle(frame, (x, y), 4, (0, 255, 0), 2)
        cv.imshow("Cam", frame)
        
term_crit = ( cv.TERM_CRITERIA_COUNT | cv.TERM_CRITERIA_EPS, 10, 1 )

cam = cv.VideoCapture(0)
cv.namedWindow('Cam')  
cv.setMouseCallback('Cam',selectROI)
frame_width = int(cam.get(3))
frame_height = int(cam.get(4))
out = cv.VideoWriter('deteccion.avi', cv.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))

while True:
   
    ret, frame = cam.read()

    if not ret:
        break
        
    if roiBox is not None:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        backProj = cv.calcBackProject([hsv], [0], roiHist, [0,180], 1)
        r, roiBox = cv.CamShift(backProj, roiBox, term_crit)
        pts = np.int0(cv.boxPoints(r))
        cv.polylines(frame, [pts], True, (0,255,0), 2)

    cv.imshow("Cam",frame)
    out.write(frame)

    k = cv.waitKey(1) & 0XFF

    if k == ord("i") and len(roiPts) < 4 :
        inputMode = True
        orig = frame.copy()

        while len(roiPts) < 4 :
            cv.imshow("Cam",frame)
            cv.waitKey(0)

        roiPts = np.array(roiPts)
        s = roiPts.sum(axis=1)
        tl = roiPts[np.argmin(s)]
        br = roiPts[np.argmax(s)]

        roi = orig[tl[1]:br[1],tl[0]:br[0]]
        roi = cv.cvtColor(roi,cv.COLOR_BGR2HSV)

        roiHist = cv.calcHist([roi], [0],None, [16], [0,180])
        roiHist = cv.normalize(roiHist, roiHist, 0, 255, cv.NORM_MINMAX)
        roiBox = (tl[0], tl[1], br[0], br[1])

    elif k == 27:
        break

        
cam.release()
out.release()
cv.destroyAllWindows()
